# Jump diffusion coefficient from a VASP file

[Previously](./vasp_msd.html), we looked at obtaining accurate estimates for the mean-squared displacement with `kinisi`. 
Here, we show using the same class to evaluate the diffusion coefficient, using the `kinisi` [methodology](./methodology.html).

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipp as sc
from kinisi.analyze import JumpDiffusionAnalyzer
from pymatgen.io.vasp import Xdatcar
np.random.seed(42)
rng = np.random.RandomState(42)

As wil the [previous example](./vasp_d.html), `params` dictionary will describe the details about the simulation.

In [ ]:
params = {'specie': 'Li',
          'time_step': 2.0 * sc.Unit('fs'),
          'step_skip': 50 * sc.Unit('dimensionless'),
          'progress': False
          }

Again, we will add an additional key-value pair to the dictionary. 
This argument means that the diffusion coefficient will only be calculated in the *xy* plane of the simulation box. 

In [ ]:
params['dimension']= 'xy'

In [ ]:
xd = Xdatcar('./example_XDATCAR.gz')
diff = JumpDiffusionAnalyzer.from_xdatcar(xd, **params)

In the above cells, we parse and determine the uncertainty on the mean-squared displacement as a function of the timestep. 
We should visualise this, to check that we are observing diffusion in our material and to determine the timescale at which this diffusion begins. 

In [ ]:
fig, ax = plt.subplots()

ax.errorbar(diff.dt.values, diff.mstd.values, np.sqrt(diff.mstd.values))
ax.set_xlabel(f'Time / {diff.dt.unit}')
ax.set_ylabel(f'MSD / {diff.mstd.unit}')
ax.set_xlim(0, None)
ax.set_ylim(0, None)
plt.show()

We can visualise this on a log-log scale, which helps to reveal the diffusive regime (the region where the gradient stops changing).

In [ ]:
fix, ax = plt.subplots()

ax.errorbar(diff.dt.values, diff.mstd.values, np.sqrt(diff.mstd.values))
ax.axvline(4000, color='g')
ax.set_xlabel(f'Time / {diff.dt.unit}')
ax.set_ylabel(f'MSD / {diff.mstd.unit}')
ax.set_yscale('log')
ax.set_xscale('log')
plt.show()

The green line at 4000 fs appears to be a reasonable estimate of the start of the diffusive regime. 
Therefore, we want to pass `4000 * sc.Units('fs')` as the argument to the diffusion analysis below. 
At this stage, we pass the `random_state` argument to ensure reproducibility. 

In [ ]:
start_of_diffusion = 4000 * sc.Unit('fs')
diff.jump_diffusion(start_of_diffusion, progress=False, random_state=rng)

This method estimates the correlation matrix between the timesteps and uses posterior sampling to find the self-diffusion coefficient, $D*$ and intercept.
We can find the mean of the marginal posterior samples of $D*$: 

In [ ]:
diff.D_J.mean()

And the standard deviation:

In [ ]:
diff.D_J.std(ddof=1)

Or we can get all of the sampled values from one of these objects. 

In [ ]:
diff.D_J.values

The same for the intercept can be found from `diff.intercept`. 
A histogram of the marginal posterior probability distribution for $D*$ can be plotted as shown below. 

Having completed the analysis, we can save the object for use later (such as downstream by a plotting script). 

These hidden cells exist to remove any existing `example.hdf` file on builiding.

In [ ]:
!rm example.hdf

In [ ]:
diff.save('example.hdf')

The data is saved in a HDF5 file format which helps us efficiently store our data. 
We can then load the data with the following class method. 

In [ ]:
loaded_diff = JumpDiffusionAnalyzer.load('example.hdf')

We can plot the data with the credible intervals from the $D$ and $D_{\text{offset}}$ distribution.

In [ ]:
credible_intervals = [[16, 84], [2.5, 97.5], [0.15, 99.85]]
alpha = [0.6, 0.4, 0.2]

fig, ax = plt.subplots()
ax.plot(loaded_diff.dt.values, loaded_diff.mstd.values, 'k-')
for i, ci in enumerate(credible_intervals):
    ax.fill_between(loaded_diff.dt.values,
                     *np.percentile(loaded_diff.distributions, ci, axis=1),
                     alpha=alpha[i],
                     color='#0173B2',
                     lw=0)
ax.set_xlabel(f'Time / {diff.dt.unit}')
ax.set_ylabel(f'MSD / {diff.msd.unit}')
ax.set_xlim(0, None)
ax.set_ylim(0, None)
plt.show()

Additionally, we can visualise the distribution of the diffusion coefficient that has been determined.

In [ ]:
fig, ax = plt.subplots()

ax.hist(loaded_diff.D_J.values, density=True)
ax.axvline(loaded_diff.D_J.mean, c='k')
ax.set_xlabel(f'D* / [{diff.D.unit}]')
ax.set_ylabel(f'p(D*) / {(1 / diff.D.unit).unit}')
plt.show()

Or the joint probability distribution for the diffusion coefficient and intercept.

In [ ]:
from corner import corner

In [ ]:
corner(np.array([i.values for i in loaded_diff.flatchain.values()]).T, 
       labels=['/'.join([k, str(v.unit)]) for k, v in loaded_diff.flatchain.items()])
plt.show()